<a href="https://colab.research.google.com/github/juliegerlings/Deep-learning-with-cats/blob/master/Copy_of_google_colab_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 3.4MB/s 


In [0]:
import tensorflow as tf
from tensorflow import keras
import datetime as dt
import os
import numpy as np
from google.colab import files
from google.colab import drive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-05-09 09:21:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.248.161, 34.206.36.121, 35.172.177.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.248.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  14.30M  41.3MB/s    in 0.3s    

2019-05-09 09:21:54 (41.3 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://40d308ae.ngrok.io


In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(256).shuffle(10000)
train_dataset = train_dataset.map(lambda x, y: (tf.div(tf.cast(x, tf.float32), 255.0), tf.reshape(tf.one_hot(y, 10), (-1, 10))))
train_dataset = train_dataset.map(lambda x, y: (tf.image.central_crop(x, 0.75), y))
train_dataset = train_dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
train_dataset = train_dataset.repeat()

In [0]:
valid_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(5000).shuffle(10000)
valid_dataset = valid_dataset.map(lambda x, y: (tf.div(tf.cast(x, tf.float32),255.0), tf.reshape(tf.one_hot(y, 10), (-1, 10))))
valid_dataset = valid_dataset.map(lambda x, y: (tf.image.central_crop(x, 0.75), y))
valid_dataset = valid_dataset.repeat()

In [0]:
def create_model():
    model = keras.models.Sequential([
        keras.layers.Conv2D(96, 3, padding='same', activation=tf.nn.relu,
                            kernel_initializer=keras.initializers.VarianceScaling(distribution='truncated_normal'),
                            kernel_regularizer=keras.regularizers.l2(l=0.001),
                            input_shape=(24, 24, 3)),
        keras.layers.Conv2D(96, 3, 2, padding='same', activation=tf.nn.relu,
                            kernel_initializer=keras.initializers.VarianceScaling(distribution='truncated_normal'),
                            kernel_regularizer=keras.regularizers.l2(l=0.001)),
        keras.layers.Dropout(0.2),
        keras.layers.Conv2D(192, 3, padding='same', activation=tf.nn.relu,
                            kernel_initializer=keras.initializers.VarianceScaling(distribution='truncated_normal'),
                            kernel_regularizer=keras.regularizers.l2(l=0.001)),
        keras.layers.Conv2D(192, 3, 2, padding='same', activation=tf.nn.relu,
                            kernel_regularizer=keras.regularizers.l2(l=0.001)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation=tf.nn.relu,
                           kernel_initializer=keras.initializers.VarianceScaling(),
                           kernel_regularizer=keras.regularizers.l2(l=0.001)),
        keras.layers.Dense(10),
        keras.layers.Softmax()
    ])

    model.compile(optimizer=tf.train.AdamOptimizer(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [0]:
model = create_model()

In [0]:
class GoogleDriveStore(keras.callbacks.Callback):
    def on_train_begin(self, logs={}, model_folder="."):
        self.first = True
        self.init_date = dt.datetime.now()
        self.model_folder = model_folder
        
        # Authenticate and create the PyDrive client.
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
        

    def on_epoch_begin(self, batch, logs={}):
        if not self.first:
          # get the latest 
          model_files = os.listdir(self.model_folder)
          max_date = self.init_date
          for f in model_files:
            if os.path.isfile(self.model_folder + "/" + f):
              if f.split(".")[-1] == 'hdf5':
                creation_date = dt.datetime.fromtimestamp(
                    os.path.getmtime((self.model_folder + "/" + f)))
                if creation_date > max_date:
                  file_name = f
                  latest_file_path = self.model_folder + "/" + f
                  max_date = creation_date
          uploaded = self.drive.CreateFile({'title': file_name})
          uploaded.SetContentFile(latest_file_path)
          uploaded.Upload()
        else:
          self.first = False

In [0]:
g_drive_callback = GoogleDriveStore()

In [0]:
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
  keras.callbacks.ModelCheckpoint("./weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=0, save_best_only=True),
  g_drive_callback
]

In [0]:
model.fit(train_dataset,  epochs=50, steps_per_epoch=len(x_train)//256,
          validation_data=valid_dataset,
          validation_steps=3, callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
195/195 [==============================] - 16s 84ms/step - loss: 2.2804 - acc: 0.3936 - val_loss: 2.1482 - val_acc: 0.4595
Epoch 2/50
195/195 [==============================] - 11s 58ms/step - loss: 1.6995 - acc: 0.5323 - val_loss: 1.7817 - val_acc: 0.5141
Epoch 3/50
195/195 [==============================] - 10s 53ms/step - loss: 1.4774 - acc: 0.5972 - val_loss: 1.3847 - val_acc: 0.6351
Epoch 4/50
195/195 [==============================] - 10s 53ms/step - loss: 1.3188 - acc: 0.6546 - val_loss: 1.3316 - val_acc: 0.6520
Epoch 5/50
195/195 [==============================] - 11s 56ms/step - loss: 1.2292 - acc: 0.6851 - val_loss: 1.2075 - val_acc: 0.6932
Epoch 6/50
195/195 [==============================] - 11s 54ms/step - loss: 1.1623 - acc: 0.7125 - val_loss: 1.2605 - val_acc: 0.6764
Epoch 7/50
195/195 [==============================] - 11s 55ms/step - loss: 1.1320 - acc: 0.7244 - val_loss: 1.1119 - val_acc: 0.7365
Epoch 8/50
195

#### One way of accessing the files on your Google Drive (mounting the drive is easier though...)

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
g_drive = GoogleDrive(gauth)
model_google_filename = "weights.12-1.05.hdf5"
file_list = g_drive.ListFile({'q': ""}).GetList()
# file_here = drive.CreateFile({'title': "downloaded_" + model_google_filename})
for file in file_list:
  if file['title'] == model_google_filename:
    file.GetContentFile("downloaded_" + model_google_filename)

In [0]:
# to demonstrate that the model loading from Google Drive actually works, first "delete" the current model
model = None
model = create_model()
model.load_weights("downloaded_" + model_google_filename)
model.predict(valid_dataset, steps=1)

OSError: ignored

In [0]:
uploaded = files.upload()

In [0]:
files.download("downloaded_weights.12-1.05.hdf5")

In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# to demonstrate that the model loading from Google Drive actually works, first "delete" the current model
# this only works if you have first mounted Google Drive to gdrive
model = None
model = create_model()
model.load_weights("./gdrive/My Drive/weights.12-1.05.hdf5")
model.predict(valid_dataset, steps=1)

array([[6.9904719e-03, 3.2309993e-04, 8.6367587e-03, ..., 2.1517063e-04,
        1.7591183e-03, 5.1133637e-04],
       [1.8606793e-02, 2.2275315e-01, 3.7019683e-05, ..., 3.3318859e-06,
        7.5644284e-01, 2.0655405e-03],
       [5.3261776e-02, 1.2857358e-01, 2.1629019e-03, ..., 1.6319279e-04,
        8.0700523e-01, 5.2667125e-03],
       ...,
       [1.5018123e-03, 6.4274087e-04, 9.9991495e-03, ..., 2.9173230e-03,
        6.3320860e-04, 1.6855212e-03],
       [2.3014853e-05, 5.8488793e-05, 3.9712328e-02, ..., 2.5212040e-02,
        1.0431206e-05, 1.3921551e-05],
       [8.1411293e-03, 6.0523057e-01, 1.3112490e-05, ..., 4.6595069e-06,
        3.5865405e-06, 3.8657606e-01]], dtype=float32)